In [6]:
from prep_data import PrepData
from anchor_boxes import *
from yolo_model import *
from predicting import *

import numpy as np
import tensorflow as tf

In [4]:
input_shape = (676, 380, 3)
grid_dim = 13
anchor_sizes = [(0.2, 0.1), (0.3, 0.2), (0.4, 0.3), (0.5, 0.4)]
num_anchors = len(anchor_sizes)

In [5]:
p = PrepData()
train, test = p.load_images()
labels = p.connect_labels(train, grid_dim, anchor_sizes, iou_vectorized)
train, test = p.preprocess_images(train), p.preprocess_images(test)

# swap axes of x and y of the image
train = np.swapaxes(train, 1, 2)
test = np.swapaxes(test, 1, 2)
train.shape

(1001, 676, 380, 3)

In [9]:
def custom_iou_loss(y_true, y_pred):
    # Compute IOUs
    ious = iou_vectorized(y_true[..., :4], y_pred[..., :4])
    iou_loss = tf.abs(ious - y_true[..., -1]) # IOU losses - the confidence scores
    
    # Mean squared error over IOU losses
    return tf.reduce_mean(tf.square(iou_loss))

def custom_iou_metric(y_true, y_pred):
    ious = iou_vectorized(y_true[..., :4], y_pred[..., :4])
    return tf.reduce_mean(ious)

In [ ]:
model = yolo_model(input_shape, grid_dim, num_anchors)
model.compile(optimizer='adam', loss=custom_iou_loss, metrics=['accuracy', custom_iou_metric])
model.fit(train, labels, epochs=3, batch_size=16, verbose=1)

In [7]:
image = test[0]
predicted_image = predict_image(image, model)
cv2.imshow('image', predicted_image)

NameError: name 'model' is not defined